# Modelo de recomendación de juegos


En esta jupyter notebook se hace la experimentación para encontrar dos modelos de recomendación, que generen una lista de 5 juegos ya sea ingresando el nombre de un juego o el id de un usuario.

En el primer caso, el modelo tiene una relación ítem-ítem, esto es se toma un juego y en base a que tan similar es ese juego con el resto de los juegos se recomiendan similares. En el segundo caso, el modelo aplicar un filtro usuario-juego, es decir, toma un usuario, encuentra usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron.

Para generar estos modelos se adoptaron algoritmos basados en la memoria, los que abordan el problema del **filtrado colaborativo** utilizando toda la base de datos, tratando de encontrar usuarios similares al usuario activo (es decir, los usuarios para los que se les quiere recomendar) y utilizando sus preferencias para predecir las valoraciones del usuario activo.

## Importaciones

In [2]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

## Datos a utilizar
Se leen los datos que se prepararon luego del EDA y se convierten en dataframe para ser utilizados por el modelo.



In [6]:
df = pd.read_csv('../Data/df_recomendacion.csv')
df

,user_id,item_name,rating
0,76561197970982479,Killing floor,5
1,76561197970982479,Zeno clash,5
2,76561197970982479,Metro 2033,5
3,js41637,Barbie™ dreamhouse party™,5
4,js41637,Euro truck simulator 2,5
...,...,...,...
52772,76561198312638244,Half-life: blue shift,5
52773,76561198312638244,Half-life,5
52774,76561198312638244,Black mesa,5
52775,LydiaMorley,Counter-strike nexon: zombies,5


In [7]:
df.tail()

,user_id,item_name,rating
52772,76561198312638244,Half-life: blue shift,5
52773,76561198312638244,Half-life,5
52774,76561198312638244,Black mesa,5
52775,LydiaMorley,Counter-strike nexon: zombies,5
52776,LydiaMorley,Counter-strike: global offensive,5


El primer paso es crear un dataframe que contiene los 'user_id' como idices, los juegos ('item_name') como columnas y como valores los `rating`.

In [7]:
piv = df.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,! that bastard is trying to steal our gold !,0rbitalis,"10,000,000",100% orange juice,1001 spikes,12 is better than 6,12 labours of hercules,12 labours of hercules ii: the cretan bull,123 slaughter me street,140,...,Zombie driver hd,Zombie exodus,Zombie kill of the week - reborn,Zombie party,Zombie shooter,Zombie zoeds,Zombies monsters robots,Zoombinis,Zuma's revenge,[the sequence]
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zynxgameth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A continuación, se normalizan los valores del dataframe `piv` restar la media de las calificaciones de un usuario y luego dividir por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario de manera que estén centradas en cero y escaladas en función de su variabilidad. A los usuarios que solo han dado una calificación o han calificado todos los juegos de la misma manera serán eliminados durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

In [8]:
# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacíos con 0 y se hace la transpuesta
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,-GM-Dragon,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,01189958889189157253,042153100,0896398616,095732,09879655452567,1011001,...,zeroepix,zerzang,ziass,zile98,zixwot,zp3413,zrustz16,zuilde,zuzuga2003,zv_odd
item_name,,,,,,,,,,,,,,,,,,,,,
! that bastard is trying to steal our gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0rbitalis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% orange juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombie zoeds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombies monsters robots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zoombinis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros. La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

In [9]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<3315x3700 sparse matrix of type '<class 'numpy.float64'>'
	with 16532 stored elements in Compressed Sparse Row format>

Ahora, se crean dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

La similitud del coseno es una medida comúnmente utilizada para evaluar la similitud entre dos vectores en un espacio multidimensional. En el contexto de sistemas de recomendación y análisis de datos, la similitud del coseno se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [10]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

Para estructurar y organizar los resultados de manera más accesible y comprensible se insertan las matrices anteriores en un Dataframe.



In [12]:
#item similarity dataframe
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
#user similarity dataframe
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)


## Función de recomendación según un juego

Ahora, conociendo la relación entre los distintos juegos, se puede proponer una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno. Esta función toma un nombre de un juego como entrada, luego ordena la columna correspondiente a ese juego en la matriz de similitud entre elementos (item_sim_df) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.

In [13]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función imprime una lista de juegos 5 similares al dado.

    '''
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in item_sim_df.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1  

In [15]:
# ejemplo
top_game('Killing floor')

Similar games to Killing floor include:

No. 1: Tales runner
No. 2: Company of heroes: tales of valor
No. 3: Fieldrunners
No. 4: Hitman: blood money
No. 5: Shadow warrior classic redux


In [16]:
# ejemplo
top_game('Barbie™ dreamhouse party™')

Similar games to Barbie™ dreamhouse party™ include:

No. 1: Return to castle wolfenstein
No. 2: Resident evil 5 / biohazard 5
No. 3: Resident evil 6 / biohazard 6
No. 4: Resident evil revelations / biohazard revelations
No. 5: Resident evil revelations 2 / biohazard revelations 2


## Función de recomendación según un usuario

La siguiente función tiene la finalidad de mostrar una lista de usuarios más similares a un usuario dado, junto con sus valores de similitud. Primero, ordena la matriz de similitud entre usuarios (user_sim_df) en orden descendente según la similitud con el usuario dado ('user_id'), luego toma los 5 usuarios más similares (excluyendo el propio usuario) y almacena sus nombres y valores de similitud en las listas sim_users y sim_values. Finalmente, combina los nombres de usuario y los valores de similitud en una lista de tuplas utilizando la función zip y los imprime.

Esto es útil en sistemas de recomendación para mostrar a un usuario los usuarios más similares en función de sus calificaciones pasadas, lo que puede ayudar en la generación de recomendaciones personalizadas.

In [17]:
def top_users(user):
    '''
    Muestra una lista de los usuarios más similares a un usuario dado y sus valores de similitud.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean encontrar usuarios similares.

    Returns:
        None: Esta función imprime la lista de usuarios similares y sus valores de similitud en la consola.

    '''
    # Verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    # Ordena los usuarios por similitud descendente y toma los 5 usuarios más similares (excluyendo el propio 'user')
    sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:6]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    # Combina los nombres de usuario y los valores de similitud en una lista de tuplas
    zipped = zip(sim_users, sim_values,)
    
    # Itera a través de las tuplas y muestra los usuarios similares y sus valores de similitud
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [23]:
# ejemplo
top_users('zeroepix')

Most Similar Users:

User #Rac00nZ, Similarity value: 0.21
User #sky_4547, Similarity value: 0.11
User #76561197993725260, Similarity value: 0.11
User #76561198070780240, Similarity value: 0.11
User #76561198065438445, Similarity value: 0.11


En esta función, se generar una lista de 5 juegos recomendados para un usuario en función de las calificaciones de usuarios similares. Los juegos que son más frecuentemente recomendados por usuarios similares se consideran como las principales recomendaciones para ese usuario.

In [24]:
def similar_user_recs(user):
    '''
    Genera una lista de los juegos más recomendados para un usuario, basándose en las calificaciones de usuarios similares.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean generar recomendaciones.

    Returns:
        list: Una lista de los juegos más recomendados para el usuario basado en la calificación de usuarios similares.

    '''
    # Verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    # Obtiene los usuarios más similares al usuario dado
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    
    # Cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Ordena los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Devuelve los 5 juegos más recomendados
    return sorted_list[:5] 

In [25]:
# ejemplo
similar_user_recs('zeroepix')

[('Tomb raider', 5),
 ('The walking dead', 2),
 ('Factorio', 2),
 ('H1z1: just survive', 1),
 ('I am bread', 1)]

In [28]:
# ejemplo
similar_user_recs('09879655452567')

[('Counter-strike: global offensive', 3),
 ('Grand theft auto v', 2),
 ('Arma 3', 1),
 ('Planetside 2', 1),
 ('War thunder', 1)]

## Carga de las matrices para recomendar en la API

Para poder utilizar las funciones top_users y similar_user_recs es necesario consumir las matrices piv_norm y user_sim_df. Por ello, se guardan, en este caso, en formato parquet los que permite una compresión y codificación eficiente.

In [30]:
pq.write_table(pa.Table.from_pandas(piv_norm), '../Data/piv_norm.parquet')
pq.write_table(pa.Table.from_pandas(user_sim_df), '../Data/user_sim_df.parquet')
pq.write_table(pa.Table.from_pandas(item_sim_df), '../Data/item_sim_df.parquet')
print('Se guardaron correctamente')

Se guardaron correctamente
